In [1]:
#for scraping booking
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
#for scraping +api
import pandas as pd
import json
import plotly.io as pio
import plotly.express as px

# ***we work on (Paris) at the begning at the end of the page we work on all***

In [2]:
r2=requests.get('https://www.booking.com/searchresults.fr.html?ss=Le+Mont-Saint-Michel&ssne=Le+Mont-Saint-Michel&ssne_untouched=Le+Mont-Saint-Michel&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAK0iaSjBsACAdICJDI0ZTRjMGQzLTkwNjctNGJkZi04MWYzLWZhODIwNjdjZDViN9gCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=900039327&dest_type=city&group_adults=2&no_rooms=1&group_children=0')
r2.text[0:200]


"<!DOCTYPE html>\n<!--\nYou know you could be getting paid to poke around in our code?\nWe're hiring designers and developers to work in Amsterdam:\nhttps://careers.booking.com/\n-->\n<!-- wdot-802 -->\n<scri"

In [3]:
# Instanciate BeautifulSoup class
soup2=BeautifulSoup( r2.text, "html.parser") # Here we used an HTML Parser

In [4]:
#loonking for hotel links
for i in soup2.find_all('a') :
  print(i.attrs)

{'href': '#skipto_main', 'class': ['a11y-skip-to-content']}
{'href': 'https://www.booking.com/index.fr.html', 'aria-label': "Booking.com réservations d'hôtels en ligne", 'data-et-click': '\nYTBUIHOdVLBLKAMZUC:1\nYTBUIHOdVLBLKAMZUC:2\n'}
{'class': ['bui-list-item', 'bui-list-item--size-small', 'js-header-language-item'], 'href': '/city/fr/le-mont-saint-michel.en-gb.html', 'data-lang': 'en-gb', 'hreflang': 'en-gb'}
{'class': ['bui-list-item', 'bui-list-item--size-small', 'js-header-language-item'], 'href': '/city/fr/le-mont-saint-michel.html', 'data-lang': 'en-us', 'hreflang': 'en-us'}
{'class': ['bui-list-item', 'bui-list-item--size-small', 'js-header-language-item'], 'href': '/city/fr/le-mont-saint-michel.es.html', 'data-lang': 'es', 'hreflang': 'es'}
{'class': ['bui-list-item', 'bui-list-item--size-small', 'js-header-language-item'], 'href': '/city/fr/le-mont-saint-michel.zh-cn.html', 'data-lang': 'zh-cn', 'hreflang': 'zh-cn'}
{'class': ['bui-list-item', 'bui-list-item--size-small', '

In [6]:
#il faut avoir +200 cell
print(len(soup2.find_all('a')) )

200


In [7]:
hello=soup2.find_all('a',attrs={'class':"bui-card__header_full_link_wrap"})[0]
print(hello)

<a class="bui-card__header_full_link_wrap" data-lp-ga-click="reviews-targeted-hotel:click-title" href="/hotel/fr/de-la-digue.fr.html">
<h3 class="bui-card__title">Hotel De La Digue</h3>
<p class="bui-card__subtitle">
Hôtel dans ce quartier : Le Mont-Saint-Michel
</p>
</a>


In [8]:
#hotel name
testotelname=hello.text.split('\n')[2]
testotelname

''

In [9]:
#test on 1 hotel  before test on all
a=str(hello).split('href=')[1].split( 'title=')[0].split('"')[1]
print(a)

/hotel/fr/de-la-digue.fr.html


In [10]:
#how many links
n=len(soup2.find_all('a',attrs={'class':"bui-card__header_full_link_wrap"}))
print("we will get",n, "links")

we will get 9 links


In [11]:
#lets get them
links=[]
names=[]

for i in range(n):
  c=soup2.find_all('a',attrs={'class':"bui-card__header_full_link_wrap"})[i]

  #split on c to get link nd put it in list
  names.append(c.text.split('\n')[2])

  links.append('https://www.booking.com'+ str(c).split('href=')[1].split( 'title=')[0].split('"')[1])  #here we add 'https://www.booking.com/' to the hrefs



print(links)
print(names)

['https://www.booking.com/hotel/fr/de-la-digue.fr.html', 'https://www.booking.com/hotel/fr/ha-el-la-croix-blanche.fr.html', 'https://www.booking.com/hotel/fr/vert.fr.html', 'https://www.booking.com/hotel/fr/le-relais-saint-michel.fr.html', 'https://www.booking.com/hotel/fr/le-relais-du-roy.fr.html', 'https://www.booking.com/hotel/fr/les-terrasses-poulard.fr.html', 'https://www.booking.com/hotel/fr/auberge-saint-pierre.fr.html', 'https://www.booking.com/hotel/fr/la-vieille-auberge-le-mont-saint-michel.fr.html', 'https://www.booking.com/hotel/fr/la-mere-poulard.fr.html']
['', '', '', '', '', '', '', '', '']


In [12]:
# #let s add www.book...
# urls=[]
# for i in listvide:
#   urls.append('https://www.booking.com/'+i)
# print(urls[0:4])

In [13]:
#we convert it into dataframe
df=pd.DataFrame()
# df['names']=names[0:3]  #on va prendre que les 3 premiers pour scraper vite apres un page par page...
df['names']=names
df['links']=links
df['lat']=np.nan
df['lon']=np.nan
df['hotel_note']=np.nan
df['description']=np.nan
display(df.head() )


,names,links,lat,lon,hotel_note,description
0,,https://www.booking.com/hotel/fr/de-la-digue.f...,NaN,NaN,NaN,NaN
1,,https://www.booking.com/hotel/fr/ha-el-la-croi...,NaN,NaN,NaN,NaN
2,,https://www.booking.com/hotel/fr/vert.fr.html,NaN,NaN,NaN,NaN
3,,https://www.booking.com/hotel/fr/le-relais-sai...,NaN,NaN,NaN,NaN
4,,https://www.booking.com/hotel/fr/le-relais-du-...,NaN,NaN,NaN,NaN


# ***now page after page***

In [14]:
for i in df['links']:
  print(i)

https://www.booking.com/hotel/fr/de-la-digue.fr.html
https://www.booking.com/hotel/fr/ha-el-la-croix-blanche.fr.html
https://www.booking.com/hotel/fr/vert.fr.html
https://www.booking.com/hotel/fr/le-relais-saint-michel.fr.html
https://www.booking.com/hotel/fr/le-relais-du-roy.fr.html
https://www.booking.com/hotel/fr/les-terrasses-poulard.fr.html
https://www.booking.com/hotel/fr/auberge-saint-pierre.fr.html
https://www.booking.com/hotel/fr/la-vieille-auberge-le-mont-saint-michel.fr.html
https://www.booking.com/hotel/fr/la-mere-poulard.fr.html


In [15]:
description=[]
hotel_note=[]
lat=[]
lon=[]
p,_=df.shape
p
for i in range(p):

  r=requests.get(df['links'][i])
  print(df['links'][i])
  soup2=BeautifulSoup(r.text,'html.parser')
#=================================================================

  #to get the decription inside the page

  hi=soup2.find('div',{'id':'property_description_content'}).text
  description.append(hi)

  # print(description)

#================================================================
#user note
  notebyuser=soup2.find('div',{'class':"b5cd09854e d10a6220b4"}).text

  hotel_note.append(notebyuser[0:3])


  print(hotel_note[i])

#================================================================
#gps work
#i get the code after looking doing this : gps=soup.find('p',{'class':"address address_clean"}) print(gps)
  t1=str(soup2.find('a',{'id':"hotel_header"})).split('bbox="')[1].split(',')[0]
  t2=str(soup2.find('a',{'id':"hotel_header"})).split('bbox="')[1].split(',')[1]

  lat.append(t1)

  lon.append(t2)

  print(lat[i])
  print(lon[i])
  #petit condition is on a pas scraper de la 1er page de recherche on recupere les info via la page de chaque hotel
  if df['names'][i]=='':
    df['names'][i]=soup2.find('h2',{'class':"d2fee87262 pp-header__title"}).text
  else:
    df['names'][i]


https://www.booking.com/hotel/fr/de-la-digue.fr.html
7,1
48.5989161328681
-1.53814339058278


<ipython-input-15-e14a69618c91>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['names'][i]=soup2.find('h2',{'class':"d2fee87262 pp-header__title"}).text


https://www.booking.com/hotel/fr/ha-el-la-croix-blanche.fr.html
7,7
48.6177686476982
-1.53709677268398
https://www.booking.com/hotel/fr/vert.fr.html
8,0
48.5967350697173
-1.53684134303157
https://www.booking.com/hotel/fr/le-relais-saint-michel.fr.html
7,9
48.5996218550758
-1.53762208153347
https://www.booking.com/hotel/fr/le-relais-du-roy.fr.html
8,0
48.5982972879393
-1.53813098690421
https://www.booking.com/hotel/fr/les-terrasses-poulard.fr.html
7,3
48.6173840090681
-1.53761423137484
https://www.booking.com/hotel/fr/auberge-saint-pierre.fr.html
7,9
48.6177225621183
-1.53711820547634
https://www.booking.com/hotel/fr/la-vieille-auberge-le-mont-saint-michel.fr.html
7,2
48.6180975834269
-1.53869289822871
https://www.booking.com/hotel/fr/la-mere-poulard.fr.html
7,1
48.6171199006609
-1.53777502133808


In [16]:
df['lat']=lat
df['description']=description
df['lon']=lon
df['hotel_note']=hotel_note

In [17]:
df[0:20]

,names,links,lat,lon,hotel_note,description
0,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.5989161328681,-1.53814339058278,"7,1",\n L'hôtel De La Digue est un établissement tr...
1,Hôtel la Croix Blanche,https://www.booking.com/hotel/fr/ha-el-la-croi...,48.6177686476982,-1.53709677268398,"7,7",\n Situé au cœur du village médiéval du Mont-S...
2,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html,48.5967350697173,-1.53684134303157,"8,0",\nVous pouvez bénéficier d'une réduction Geniu...
3,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.5996218550758,-1.53762208153347,"7,9",\nVous pouvez bénéficier d'une réduction Geniu...
4,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,48.5982972879393,-1.53813098690421,"8,0",\nVous pouvez bénéficier d'une réduction Geniu...
5,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.6173840090681,-1.53761423137484,"7,3",\nVous pouvez bénéficier d'une réduction Geniu...
6,Auberge Saint Pierre,https://www.booking.com/hotel/fr/auberge-saint...,48.6177225621183,-1.53711820547634,"7,9",\n L'Auberge Saint-Pierre occupe une maison à ...
7,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,48.6180975834269,-1.53869289822871,"7,2",\n La Vieille Auberge vous accueille dans le v...
8,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,48.6171199006609,-1.53777502133808,"7,1",\nVous pouvez bénéficier d'une réduction Geniu...


In [18]:
len(df['description'].unique())  #

9

In [19]:
df.isnull().any()

names          False
links          False
lat            False
lon            False
hotel_note     False
description    False
dtype: bool

In [21]:
df['lat'][1][0:5]

'48.61'

In [22]:
df['description']=df['description'].apply(lambda x : x.replace('\n','') ) #les \n au debut deleted
df['lat']=df['lat'].apply(lambda x : df['lat'][1][0:5])  # just 2 number after .
df['lon']=df['lon'].apply(lambda x : df['lon'][1][0:5] )

In [23]:
df['lat']=pd.to_numeric(df['lat'])
df['lon']=pd.to_numeric(df['lon'])
type(df['lat'][0])

numpy.float64

In [25]:
df['hotel_note']=df['hotel_note'].apply(lambda x : x.replace(",",'.') )

In [26]:
df['hotel_note']=pd.to_numeric(df['hotel_note'])

In [27]:
df.head()

,names,links,lat,lon,hotel_note,description
0,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.61,-1.53,7.1,L'hôtel De La Digue est un établissement trad...
1,Hôtel la Croix Blanche,https://www.booking.com/hotel/fr/ha-el-la-croi...,48.61,-1.53,7.7,Situé au cœur du village médiéval du Mont-Sai...
2,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html,48.61,-1.53,8.0,Vous pouvez bénéficier d'une réduction Genius ...
3,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.61,-1.53,7.9,Vous pouvez bénéficier d'une réduction Genius ...
4,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,48.61,-1.53,8.0,Vous pouvez bénéficier d'une réduction Genius ...


In [28]:
df.to_csv('bookin1exemple.csv', index=False, encoding='utf-8')

In [29]:
fig = px.scatter_mapbox(df, lat="lat",
                        lon="lon",
                        color="hotel_note",
                        zoom=10,
                        mapbox_style="carto-positron")
fig.show()

In [ ]:
dic={
"Paris":'https://www.booking.com/searchresults.fr.html?ss=Paris&ssne=Paris&ssne_untouched=Paris&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKIxpijBsACAdICJDAwNmExMDRkLTZkMmEtNDA5Yy1iNzNlLWM0N2RhMjQxZTViOdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1456928&dest_type=city&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure',
"Mont Saint Michel":'https://www.booking.com/searchresults.fr.html?ss=Le+Mont-Saint-Michel&ssne=Le+Mont-Saint-Michel&ssne_untouched=Le+Mont-Saint-Michel&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKo8qOjBsACAdICJGRmY2FkMjhkLTY1MWUtNGE2Ny1hMTg5LTQzNDhlNjc1YmEyZNgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=900039327&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
"St Malo":'https://www.booking.com/searchresults.fr.html?ss=St+Malo&ssne=Saint-Malo&ssne_untouched=Saint-Malo&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKT86OjBsACAdICJDBlZjAxY2Y4LTc4OWQtNDFjZC1hNWQ4LTVhMWM1MTliM2NjM9gCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1466824&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=fr&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=22f37609fbe50123&ac_meta=GhAyMmYzNzYwOWZiZTUwMTIzIAAoATICZnI6B1N0IE1hbG9AAEoAUAA%3D&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure',
"Bayeux":'https://www.booking.com/searchresults.fr.html?ss=Bayeux&ssne=Bayeux&ssne_untouched=Bayeux&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuALw86OjBsACAdICJDFjMGE2N2Q3LTRhOTYtNGFjNC1iNjBlLWY3Mzg5NzEzMDVkZdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1410836&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
"Le Havre":'https://www.booking.com/searchresults.fr.html?ss=Le+Havre&ssne=Le+Havre&ssne_untouched=Le+Havre&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuALw86OjBsACAdICJDFjMGE2N2Q3LTRhOTYtNGFjNC1iNjBlLWY3Mzg5NzEzMDVkZdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1441598&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
}

# "Amiens",
# "Lille",
# "Strasbourg",
# "Chateau du Haut Koenigsbourg",
# "Colmar",
# "Eguisheim",
# "Besancon",
# "Dijon",
# "Annecy",
# "Grenoble",
# "Lyon",
# "Gorges du Verdon",
# "Bormes les Mimosas",
# "Cassis",
# "Marseille",
# "Aix en Provence",
# "Avignon",
# "Uzes",
# "Nimes",
# "Aigues Mortes",
# "Saintes Maries de la mer",
# "Collioure",
# "Carcassonne",
# "Ariege",
# "Toulouse",
# "Montauban",
# "Biarritz",
# "Bayonne",
# "La Rochelle"}

In [38]:
urls=['https://www.booking.com/searchresults.fr.html?ss=Paris&ssne=Paris&ssne_untouched=Paris&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKIxpijBsACAdICJDAwNmExMDRkLTZkMmEtNDA5Yy1iNzNlLWM0N2RhMjQxZTViOdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1456928&dest_type=city&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure','https://www.booking.com/searchresults.fr.html?ss=Le+Havre&ssne=Le+Havre&ssne_untouched=Le+Havre&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuALw86OjBsACAdICJDFjMGE2N2Q3LTRhOTYtNGFjNC1iNjBlLWY3Mzg5NzEzMDVkZdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1441598&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
      'https://www.booking.com/searchresults.fr.html?ss=Bayeux&ssne=Bayeux&ssne_untouched=Bayeux&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuALw86OjBsACAdICJDFjMGE2N2Q3LTRhOTYtNGFjNC1iNjBlLWY3Mzg5NzEzMDVkZdgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1410836&dest_type=city&group_adults=2&no_rooms=1&group_children=0',
      'https://www.booking.com/searchresults.fr.html?ss=St+Malo&ssne=Saint-Malo&ssne_untouched=Saint-Malo&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKT86OjBsACAdICJDBlZjAxY2Y4LTc4OWQtNDFjZC1hNWQ4LTVhMWM1MTliM2NjM9gCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1466824&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=fr&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=22f37609fbe50123&ac_meta=GhAyMmYzNzYwOWZiZTUwMTIzIAAoATICZnI6B1N0IE1hbG9AAEoAUAA%3D&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure',
      'https://www.booking.com/searchresults.fr.html?ss=Le+Mont-Saint-Michel&ssne=Le+Mont-Saint-Michel&ssne_untouched=Le+Mont-Saint-Michel&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaOIBiAEBmAENuAEGyAEM2AEB6AEBiAIBqAIDuAKo8qOjBsACAdICJGRmY2FkMjhkLTY1MWUtNGE2Ny1hMTg5LTQzNDhlNjc1YmEyZNgCBeACAQ&sid=25116a3e813b55f0478b91d5b7063890&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=900039327&dest_type=city&group_adults=2&no_rooms=1&group_children=0',]

In [39]:
df2 = pd.DataFrame()

for url in urls:
    r2 = requests.get(url)
    r2.text[0:200]
    soup2 = BeautifulSoup(r2.text, "html.parser")
    links = []
    names = []
    n = len(soup2.find_all('a', attrs={'class': "bui-card__header_full_link_wrap"}))

    for i in range(n):
        try:
            c = soup2.find_all('a', attrs={'class': "bui-card__header_full_link_wrap"})[i]
            names.append(c.text.split('\n')[2])
            links.append('https://www.booking.com' + str(c).split('href=')[1].split('title=')[0].split('"')[1])
        except IndexError:
            raise IndexError("Index out of range error occurred. Please check the HTML structure or update the code.")

    df = pd.DataFrame()
    df['names'] = names
    df['links'] = links
    df['lat'] = np.nan
    df['lon'] = np.nan
    df['hotel_note'] = np.nan
    df['description'] = np.nan

    description = []
    hotel_note = []
    lat = []
    lon = []
    p, _ = df.shape

    for i in range(p):
        r = requests.get(df['links'][i])

        display(df['links'][i])
        print('--------------------------------------------------------------------------------------------------------------------------------')

        soup2 = BeautifulSoup(r.text, 'html.parser')
        hi = soup2.find('div', {'id': 'property_description_content'}).text
        description.append(hi)
        notebyuser = soup2.find('div', {'class': "b5cd09854e d10a6220b4"}).text
        hotel_note.append(notebyuser[0:3])
        t1 = str(soup2.find('a', {'id': "hotel_header"})).split('bbox="')[1].split(',')[0]
        t2 = str(soup2.find('a', {'id': "hotel_header"})).split('bbox="')[1].split(',')[1]
        lat.append(t1)
        lon.append(t2)
        # print(lat[i]) #at the end we will have almost the  same lat nd lon bcs we did arround
        if df['names'][i] == '':
            df['names'][i] = soup2.find('h2', {'class': "d2fee87262 pp-header__title"}).text
        else:
            df['names'][i]

    df['lat'] = lat
    df['description'] = description
    df['lon'] = lon
    df['hotel_note'] = hotel_note
    df['description'] = df['description'].apply(lambda x: x.replace('\n', ''))  # delete \n
    df['lat'] = df['lat'].apply(lambda x: df['lat'][1][0:5])  #  2 decimal places
    df['lon'] = df['lon'].apply(lambda x: df['lon'][1][0:5])
    df['lat'] = pd.to_numeric(df['lat'])
    df['lon'] = pd.to_numeric(df['lon'])
    df['hotel_note'] = df['hotel_note'].apply(lambda x: x.replace(",", '.'))  # replace comma with dot
    df['hotel_note'] = pd.to_numeric(df['hotel_note'])
    print("Data is ready, moving on to the next one")
    print('===============================================================================================================================')
    df2 = pd.concat([df2, df])
    display(df)

print("Final Data:")
display(df2)


'https://www.booking.com/hotel/fr/basile.fr.html'

--------------------------------------------------------------------------------------------------------------------------------


'https://www.booking.com/hotel/fr/de-roubaix.fr.html'

--------------------------------------------------------------------------------------------------------------------------------


'https://www.booking.com/hotel/fr/hotel-le-presbytere.fr.html'

...

'https://www.booking.com/hotel/fr/la-mere-poulard.fr.html'

--------------------------------------------------------------------------------------------------------------------------------
Data is ready, moving on to the next one


,names,links,lat,lon,hotel_note,description
0,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.61,-1.53,7.1,L'hôtel De La Digue est un établissement trad...
1,Hôtel la Croix Blanche,https://www.booking.com/hotel/fr/ha-el-la-croi...,48.61,-1.53,7.7,Situé au cœur du village médiéval du Mont-Sai...
2,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html,48.61,-1.53,8.0,Vous pouvez bénéficier d'une réduction Genius ...
3,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.61,-1.53,7.9,Vous pouvez bénéficier d'une réduction Genius ...
4,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,48.61,-1.53,8.0,Vous pouvez bénéficier d'une réduction Genius ...
5,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.61,-1.53,7.3,Vous pouvez bénéficier d'une réduction Genius ...
6,Auberge Saint Pierre,https://www.booking.com/hotel/fr/auberge-saint...,48.61,-1.53,7.9,L'Auberge Saint-Pierre occupe une maison à co...
7,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,48.61,-1.53,7.2,La Vieille Auberge vous accueille dans le vil...
8,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,48.61,-1.53,7.1,Vous pouvez bénéficier d'une réduction Genius ...


Final Data:


,names,links,lat,lon,hotel_note,description
0,Le Basile Hôtel,https://www.booking.com/hotel/fr/basile.fr.html,48.84,2.325,9.0,Vous pouvez bénéficier d'une réduction Genius ...
1,Hôtel de Roubaix,https://www.booking.com/hotel/fr/de-roubaix.fr...,48.84,2.325,9.0,Vous pouvez bénéficier d'une réduction Genius ...
2,Hôtel Le Presbytère,https://www.booking.com/hotel/fr/hotel-le-pres...,48.84,2.325,9.3,Vous pouvez bénéficier d'une réduction Genius ...
3,Le 12 Hôtel,https://www.booking.com/hotel/fr/hotel-le-12.f...,48.84,2.325,9.0,Vous pouvez bénéficier d'une réduction Genius ...
4,Hotel Malte - Astotel,https://www.booking.com/hotel/fr/malteopera.fr...,48.84,2.325,9.1,L’Hotel Malte - Astotel est implanté dans un ...
...,...,...,...,...,...,...
4,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,48.61,-1.530,8.0,Vous pouvez bénéficier d'une réduction Genius ...
5,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.61,-1.530,7.3,Vous pouvez bénéficier d'une réduction Genius ...
6,Auberge Saint Pierre,https://www.booking.com/hotel/fr/auberge-saint...,48.61,-1.530,7.9,L'Auberge Saint-Pierre occupe une maison à co...
7,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,48.61,-1.530,7.2,La Vieille Auberge vous accueille dans le vil...


In [74]:
df2.to_csv('bookingfinal.csv', index=False, encoding='utf-8')

In [70]:
# Group  by city and select the top hotels
top_hotels = df2.groupby(['lat','lon']).apply(lambda x: x.nlargest(5, 'hotel_note')).reset_index(drop=True)
top_hotels

,names,links,lat,lon,hotel_note,description
0,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html,48.61,-1.530,8.0,Vous pouvez bénéficier d'une réduction Genius ...
1,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,48.61,-1.530,8.0,Vous pouvez bénéficier d'une réduction Genius ...
2,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.61,-1.530,7.9,Vous pouvez bénéficier d'une réduction Genius ...
3,Auberge Saint Pierre,https://www.booking.com/hotel/fr/auberge-saint...,48.61,-1.530,7.9,L'Auberge Saint-Pierre occupe une maison à co...
4,Hôtel la Croix Blanche,https://www.booking.com/hotel/fr/ha-el-la-croi...,48.61,-1.530,7.7,Situé au cœur du village médiéval du Mont-Sai...
5,Hôtel Le Nautilus,https://www.booking.com/hotel/fr/le-nautilus.f...,48.62,-2.030,9.1,L'Hôtel Le Nautilus occupe un bâtiment rénové...
6,Hôtel La Villefromoy,https://www.booking.com/hotel/fr/lavillefromoy...,48.62,-2.030,8.9,Vous pouvez bénéficier d'une réduction Genius ...
7,Hôtel Le Nouveau Monde,https://www.booking.com/hotel/fr/ha-tel-le-nou...,48.62,-2.030,8.8,L'Hôtel Le Nouveau Monde est situé en bord de...
8,Hôtel L'Adresse,https://www.booking.com/hotel/fr/adresse.fr.html,48.62,-2.030,8.7,Vous pouvez bénéficier d'une réduction Genius ...
9,Hôtel Bristol Union Intra Muros,https://www.booking.com/hotel/fr/bristol-union...,48.62,-2.030,8.7,L’Hôtel Bristol Union Intra Muros est situé a...


## ***Map***

In [73]:
# Displaying a map with the Top-20 hotels for the Top-5 cities
fig = px.scatter_mapbox(top_hotels,
                        title = 'best hotels in France (4 cities) ',
                        lat = 'lat',
                        lon = 'lon',
                        color = 'hotel_note',
                        hover_name="names",
                        size='hotel_note',
                        zoom=6,
                        mapbox_style="carto-positron")
fig.show()



# ***AWS time***

In [ ]:
import boto3

# aws credentials
ACCESS_KEY = "XXXX"
SECRET_KEY = "XXXX"

# create S3 session
session = boto3.Session(aws_access_key_id=ACCESS_KEY,
                        aws_secret_access_key=SECRET_KEY)

In [ ]:
# create bucket
s3 = session.resource("s3")
bucket = s3.create_bucket(Bucket="bucket-iheb-projet-kayak-wml")


In [ ]:
# Upload csv file to the bucket
put_object = bucket.put_object(Key="/content/bookingfinal.csv", Body='csv')

In [ ]:
# Download from s3 bucket
bucket.download_file('dataset-kayak.csv', 'dataset-kayak.csv')

# ***RDS Instance***

In [ ]:
# Importing libraries and modules
import psycopg2
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker